In [ ]:
import otoole
print(otoole.__version__)
from otoole import read
import os
import sys
sys.path.append('../')

# Suppress FutureWarning messages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from otoole.utils import (
    _read_file,
)
import xarray as xr

import logging

#from tzmosys.variables import add_variables

from tz.osemosys.model.variables.activity import *
from tz.osemosys.model.variables.capacity import *
from tz.osemosys.model.variables.demand import *
from tz.osemosys.model.variables.emissions import *
from tz.osemosys.model.variables.other import *
from tz.osemosys.model.variables.storage import *

from tz.osemosys.model.constraints.demand import *
from tz.osemosys.model.constraints.capacity_adequacy_a import *
from tz.osemosys.model.constraints.capacity_adequacy_b import *
from tz.osemosys.model.constraints.energy_balance_a import *
from tz.osemosys.model.constraints.energy_balance_b import *
from tz.osemosys.model.constraints.accounting_technology import *
from tz.osemosys.model.constraints.capital_costs import *
from tz.osemosys.model.constraints.operating_costs import *
from tz.osemosys.model.constraints.total_discounted_costs import *
from tz.osemosys.model.constraints.total_capacity import *
from tz.osemosys.model.constraints.new_capacity import *
from tz.osemosys.model.constraints.annual_activity import *
from tz.osemosys.model.constraints.total_activity import *
from tz.osemosys.model.constraints.salvage_value import *
from tz.osemosys.model.constraints.reserve_margin import *
from tz.osemosys.model.constraints.re_targets import *
from tz.osemosys.model.constraints.emissions import *
from tz.osemosys.model.constraints.storage import *

logger = logging.getLogger(__name__)

In [ ]:
model_name = 'otoole-full-electricity-storage'
config_path = os.path.join('../examples/otoole_compat/config_files', f'{model_name}.yaml')
folder_path = os.path.join('../examples/otoole_compat/input_csv', model_name)

print(config_path, folder_path)

In [ ]:
with open(config_path, "r") as config_file:
    config = _read_file(config_file, '.yaml')

model, defaults = read(config_path, 'csv', folder_path)

# Reshape the CSV files to create the dataset of sets and parameters
# If the sparse package is available (Python <3.11)
# data_vars = {x: xr.DataArray().from_series(y.VALUE, sparse=True) for x, y in model.items() if config[x]['type'] == 'param'}
data_vars = {x: y.VALUE.to_xarray() for x, y in model.items() if config[x]['type'] == 'param'}
coords = {x: y.values.T[0] for x, y in model.items() if config[x]['type'] == 'set'}
ds = xr.Dataset(data_vars=data_vars, coords=coords)
ds = ds.assign_coords({'_REGION': model['REGION'].values.T[0]})

for param, default in defaults.items():
    if config[param]['type'] == 'param':
        ds[param].attrs['default'] = default
        if default != 0:
            ds[param] = ds[param].fillna(default)
# ds.to_netcdf(f'{model_name}.nc', engine='netcdf4')
# ds = xr.open_dataset(f'{model_name}.nc')
# ds = ds.drop_sel(YEAR=range(2025, 2071))

# Model Creation

In [ ]:
# client = Client()
# client

In [ ]:
from linopy import Model, solvers, available_solvers

chunks = {
    'YEAR': ds.YEAR.size,
    'TIMESLICE': ds.TIMESLICE.size,
    'REGION': 1,
    'TECHNOLOGY': 100,
    'FUEL': 100,
    'MODE_OF_OPERATION': ds.MODE_OF_OPERATION.size,
    'SEASON': ds.SEASON.size
}

m = Model(force_dim_names=True) #, chunk='auto')

## Variables

In [ ]:
#m = add_variables(ds, m)

m = add_activity_variables(ds, m)
m = add_capacity_variables(ds, m)
m = add_demand_variables(ds, m)
m = add_emission_variables(ds, m)
m = add_cost_variables(ds, m)
m = add_storage_variables(ds, m)

## Discounting

```ampl
param DiscountRate{r in REGION};
param DiscountRateIdv{r in REGION, t in TECHNOLOGY}, default DiscountRate[r];

param DiscountFactor{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMid{r in REGION, y in YEAR} :=
	(1 + DiscountRate[r]) ^ (y - min{yy in YEAR} min(yy) + 0.5);

param OperationalLife{r in REGION, t in TECHNOLOGY};

param CapitalRecoveryFactor{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRateIdv[r,t])^(-1))/(1 - (1 + DiscountRateIdv[r,t])^(-(OperationalLife[r,t])));
param PvAnnuity{r in REGION, t in TECHNOLOGY} :=
	(1 - (1 + DiscountRate[r])^(-(OperationalLife[r,t]))) * (1 + DiscountRate[r]) / DiscountRate[r];

param DiscountRateStorage{r in REGION, s in STORAGE};
param DiscountFactorStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.0);
param DiscountFactorMidStorage{r in REGION, s in STORAGE, y in YEAR} :=
	(1 + DiscountRateStorage[r, s]) ^ (y - min{yy in YEAR} min(yy) + 0.5);
```

In [ ]:
discount_factor = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid = ((1 + ds['DiscountRate']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

discount_factor_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR'])))
discount_factor_mid_idv = ((1 + ds['DiscountRateIdv']) ** (ds.coords['YEAR'] - min(ds.coords['YEAR']) + 0.5))

pv_annuity = (1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife']))) * (1 + ds['DiscountRateIdv']) / ds['DiscountRateIdv']

capital_recovery_factor = (1 - (1 + ds['DiscountRateIdv'])**(-1))/(1 - (1 + ds['DiscountRateIdv'])**(-(ds['OperationalLife'])))

# Constraints

## Storage

In [ ]:
m = add_storage_constraints(ds, m)

## Demand

In [ ]:
m = add_demand_constraints(ds, m)

## Capacity Adequacy A

In [ ]:
m = add_capacity_adequacy_a_constraints(ds, m)

## Capacity Adequacy B

In [ ]:
m = add_capacity_adequacy_b_constraints(ds, m)

## Energy Balance A

In [ ]:
m = add_energy_balance_a_constraints(ds, m)

## Energy Balance B

In [ ]:
m = add_energy_balance_b_constraints(ds, m)

## Accounting Technology Production/Use

In [ ]:
m = add_accounting_technology_constraints(ds, m)

## Capital Costs

In [ ]:
m = add_capital_costs_constraints(ds, m)

## Salvage Value

In [ ]:
m = add_salvage_value_constraints(ds, m)

## Operating Costs

In [ ]:
m = add_operating_costs_constraints(ds, m)

## Total Discounted Costs

In [ ]:
m = add_total_discounted_costs_constraints(ds, m)

## Total Capacity Constraints

In [ ]:
m = add_total_capacity_constraints(ds, m)

## New Capacity Constraints

In [ ]:
m = add_new_capacity_constraints(ds, m)

## Annual Activity Constraints

In [ ]:
m = add_annual_activity_constraints(ds, m)

## Total Activity Constraints

In [ ]:
m = add_total_activity_constraints(ds, m)

## Reserve Margin Constraints

In [ ]:
m = add_reserve_margin_constraints(ds, m)

In [ ]:
m.constraints['RM3_ReserveMargin_Constraint']

## RE Production Target NTS: Should change demand for production

In [ ]:
m = add_re_targets_constraints(ds, m)

## Emissions

In [ ]:
m = add_emissions_constraints(ds, m)

# Objective Function
```ampl
minimize cost: sum{r in REGION, y in YEAR} TotalDiscountedCost[r,y];
```

In [ ]:
objective = m['TotalDiscountedCost'].sum(dims=['REGION', 'YEAR'])
m.add_objective(expr=objective, overwrite=True)

# Solving

In [ ]:
results_path = f"../results/{model_name}/"
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [ ]:
m.solve(solver_name='cbc', log_fn=f'../results/{model_name}/solver.log')

In [ ]:
results_path = f"../results/{model_name}/csv/"
if not os.path.exists(results_path):
    os.makedirs(results_path)
    
for variable, data in m.solution.items():
    data = data.to_dataframe(name=variable)
    data[data[variable] != 0.0].to_csv(f"../results/{model_name}/csv/{variable}.csv")